# Agenda, day 2

1. Recap
2. Q&A
3. Magic methods
4. Class attributes
5. Finding attributes with ICPO
6. Inheritance -- and how ICPO influences it
7. Three models for method inheritance
8. Data inheritance
9. AMA / where to from here?

# Recap

How do we solve problems? Many times, in the computer world, it's easiest to solve problems if we have data structures that are aligned with our thinking. By creating custom data structures, on top of the ones that Python provides, we can (a) write code more easily, (b) understand/maintain code more easily, and even (c) get better performance out of our code.

By doing this, we make our code easier to write and maintain. 